# Тестовое задание, Рогачев Максим Дмитриевич, 12.09.2023

In [218]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

## Часть 1. Ставка финансирования

Скрипт для выгрузки в файле get_funding_fee.py. Теперь данные выгружаются не для конкретных суток, а просто за прошедшие сутки на момент запуска. C учётом того, что Вы не сможете запустить скрипт get_borrow_interest.py на своём компьютере, запускать get_funding_fee.py нет смысла, хотя это возможно, так как он не привязан к API.

Для примера: запускаем скрипт в 10:33 16.09.23. Тогда данные выгружаются для периода 08:00 15.09.23 - 08:00 16.09.23.

Для каждого инструмента выгружаются ставки финансирования зя прошедшие сутки в столбце *fundingRate*:

In [219]:
funding_fee = pd.read_csv('funding_fees_raw.csv')
funding_fee

,symbol,fundingTime,fundingRate
0,SUSHIUSDT,1694764800000,-0.000038
1,SUSHIUSDT,1694793600000,0.000091
2,SUSHIUSDT,1694822400000,0.000100
3,BTSUSDT,1694764800000,0.000100
4,BTSUSDT,1694793600000,0.000100
...,...,...,...
622,CTSIUSDT,1694793600000,0.000100
623,CTSIUSDT,1694822400000,-0.000277
624,ACHUSDT,1694764800000,0.000100
625,ACHUSDT,1694793600000,-0.000299


Так как ставки взимаются в равные промежутки, чтобы посчитать дневной процент достаточно просто их сложить и перевести в проценты:

In [220]:
funding_fee_day = funding_fee.groupby(['symbol'])['fundingRate'].sum() * 100

Придадим читаемый и красивый вид таблице и добавим годовые проценты:

In [221]:
funding_fee_day_df = funding_fee_day.to_frame(name='ff_percents_day')
funding_fee_day_df.reset_index(inplace=True)
funding_fee_day_df['symbol'] = funding_fee_day_df['symbol'].apply(lambda x: x[0:-4])
funding_fee_day_df['ff_percents_year'] = funding_fee_day_df['ff_percents_day'] * 365


В итоговой таблице для каждого инструмента в столбце *ff_percents_day* указана дневная ставка, в столбце *ff_percents_year* указана годовая ставка:

In [222]:
funding_fee_day_df

,symbol,ff_percents_day,ff_percents_year
0,1000FLOKI,0.030000,10.950000
1,1000LUNC,0.011972,4.369780
2,1000PEPE,0.030000,10.950000
3,1000SHIB,0.010977,4.006605
4,1000XEC,0.030000,10.950000
...,...,...,...
204,YGG,0.020401,7.446365
205,ZEC,0.004996,1.823540
206,ZEN,0.020731,7.566815
207,ZIL,0.030000,10.950000


## Часть 2. Маржинальная ставка

Скрипт get_borrow_interest.py выгружает данные о маржинальной процентной ставке для каждого инструмента, доступного для торговли. Так как к API ключу имеет только мой IP адрес(иначе нельзя получить доступ к маржинальным данным), и из файла config.ini удалён секретный ключ, то вы не сможете его запустить. С учётом того, что ставка меняется в течение дня непредсказуемое число раз, данные требуют предобработки.

In [223]:
borrow_interest = pd.read_csv('borrow_interest_raw.csv')

Сейчас данные выглядят так: для каждого тикера выгружены все изменения ставки в пределах суток. Ставка меняется неограниченное число раз, поэтому для каждого инструмента может быть несколько записей. Моя задача - рассчитать суммарный dailyInterestRate для суток с учётом изменения ставки. 

In [224]:
borrow_interest

,asset,timestamp,dailyInterestRate,vipLevel
0,BTC,1694872800000,0.000044,1
1,BTC,1694822400000,0.000044,1
2,BTC,1694808000000,0.000044,1
3,BTC,1694793600000,0.000045,1
4,BTC,1694782800000,0.000044,1
...,...,...,...,...
4974,CYBER,1694779200000,0.000315,1
4975,CYBER,1694772000000,0.000332,1
4976,CYBER,1694761200000,0.000324,1
4977,CYBER,1694754000000,0.000319,1


Удаляем все записи, выходящие за пределы суток:

In [227]:
borrow_interest = borrow_interest[borrow_interest['timestamp'] <= (funding_fee.loc[2, 'fundingTime'] + 8*3600*1000)]

Добавляем для каждого тикера запись с пороговыми значениями времени, это нужно для дальнейшего расчета:

In [228]:
temp = borrow_interest[borrow_interest['timestamp'] <= funding_fee.loc[0, 'fundingTime']].groupby(['asset'])['dailyInterestRate'].first()

for asset in borrow_interest['asset'].unique():
    borrow_interest = borrow_interest.append({'asset': asset, 'timestamp':  funding_fee.loc[0, 'fundingTime'], 'dailyInterestRate': temp[asset], 'vipLevel': 1}, ignore_index=True)
    borrow_interest = borrow_interest.append({'asset': asset, 'timestamp': funding_fee.loc[2, 'fundingTime'] + 8*3600*1000, 'dailyInterestRate': 0, 'vipLevel': 1}, ignore_index=True)

borrow_interest = borrow_interest.drop_duplicates(['asset', 'timestamp'], keep='last')   
borrow_interest = borrow_interest[borrow_interest['timestamp'] >= funding_fee.loc[0, 'fundingTime']]

Сортируем записи по тикеру и времени изменения ставки:

In [229]:
borrow_interest = borrow_interest.sort_values(['asset', 'timestamp'], ascending=(False, False))

Для примера, так сейчас выглядят данные по биткоину: первая запись привязана к дате последнего изменения ставки финансирования внутри прошедших суток, процентная ставка там 0 для удобства расчётов. Последняя запись привязана к дате первого изменения ставки финансирования и ставка там 0.000043(обратите внимание, что ставка на самом деле представлена более точной дробью, но из - за особенностей вывода отображается неполное число). Далее необходимо рассчитать сколько часов конкретная ставка имела место, затем умножить полученное количество часов на ставку, поделив на 24. В итоге, если провести такие вычисления для каждой ставки внутри одного токена, то мы получим итоговую дневную ставку.

In [230]:
borrow_interest[borrow_interest['asset'] == 'BTC']

,asset,timestamp,dailyInterestRate,vipLevel
4280,BTC,1694851200000,0.000000,1
0,BTC,1694822400000,0.000044,1
1,BTC,1694808000000,0.000044,1
2,BTC,1694793600000,0.000045,1
3,BTC,1694782800000,0.000044,1
4,BTC,1694779200000,0.000042,1
5,BTC,1694768400000,0.000044,1
4279,BTC,1694764800000,0.000043,1


Здесь проводятся вышеописанные вычисления. Для каждой ставки рассчитываем количество часов, перемножаем часы со ставками и складываем, все внутри одного тикера. В этом же разделе переводим ставку в проценты:

In [231]:
borrow_interest['hours'] = (borrow_interest['timestamp'].shift(1) - borrow_interest['timestamp']) // 3600000
borrow_interest['hours*rate'] = borrow_interest['hours'] * borrow_interest['dailyInterestRate']
borrow_interest_day = borrow_interest.groupby(['asset'])['hours*rate'].sum() / 24 * 100

In [232]:
borrow_interest_day_df = borrow_interest_day.to_frame(name='bi_percents_day')
borrow_interest_day_df.reset_index(inplace=True)
borrow_interest_day_df = borrow_interest_day_df.rename(columns={'asset': 'symbol'})
borrow_interest_day_df['bi_percents_year'] = borrow_interest_day_df['bi_percents_day']*365

В конечном итоге имеем такую таблицу. Для каждого инструмента в столбце *bi_percents_day* указана итоговая ставка в процентах с учётом её изменения за сутки, а в столбце *bi_percents_year* указана годовая ставка:

In [233]:
borrow_interest_day_df

,symbol,bi_percents_day,bi_percents_year
0,1INCH,0.134970,49.263898
1,AAVE,0.020545,7.498910
2,ACH,0.278668,101.713790
3,ADA,0.020538,7.496507
4,AGIX,0.022874,8.349177
...,...,...,...
243,YGG,0.384682,140.409036
244,ZEC,0.015272,5.574113
245,ZEN,0.037569,13.712533
246,ZIL,0.086073,31.416630


## Часть 3. Поиск потенциально прибыльных монет

Следующий шаг - объединить таблицы по тикерам. В конечном итоге остаётся 180 тикеров, для каждого из которых известны дневной/годовой процент по маржинальной сделке и дневной/годовой процент финансирования по фьючерсной сделке:

In [234]:
all_data = borrow_interest_day_df.merge(funding_fee_day_df, on=['symbol'])
all_data

,symbol,bi_percents_day,bi_percents_year,ff_percents_day,ff_percents_year
0,1INCH,0.134970,49.263898,0.011613,4.238745
1,AAVE,0.020545,7.498910,0.028240,10.307600
2,ACH,0.278668,101.713790,0.002455,0.896075
3,ADA,0.020538,7.496507,0.004997,1.823905
4,AGIX,0.022874,8.349177,0.011149,4.069385
...,...,...,...,...,...
175,YGG,0.384682,140.409036,0.020401,7.446365
176,ZEC,0.015272,5.574113,0.004996,1.823540
177,ZEN,0.037569,13.712533,0.020731,7.566815
178,ZIL,0.086073,31.416630,0.030000,10.950000


Так как мы лонгуем фьючерсы и шортим маржу, то нас интересуют инструменты с отрицательной ставкой финансирования(иначе платить придётся нам):

In [235]:
all_data = all_data[all_data['ff_percents_day'] < 0].reset_index()
all_data

,index,symbol,bi_percents_day,bi_percents_year,ff_percents_day,ff_percents_year
0,11,APE,0.140858,51.413170,-0.012857,-4.692805
1,13,APT,0.041131,15.012921,-0.008368,-3.054320
2,18,ASTR,0.042894,15.656401,-0.044984,-16.419160
3,23,AXS,0.152473,55.652523,-0.186250,-67.981250
4,27,BCH,0.201755,73.640393,-0.040770,-14.881050
5,29,BLZ,2.116762,772.618221,-1.368237,-499.406505
6,30,BNB,0.260285,95.004025,-0.005152,-1.880480
7,42,COMP,0.694516,253.498294,-0.001393,-0.508445
8,46,CTSI,0.141106,51.503660,-0.007714,-2.815610
9,49,DASH,0.061077,22.293196,-0.000014,-0.005110


Оставляем только те инструменты, у которых funding_fee по модулю превышает borrow_interest в рамках суток и рассчитываем дневной и годовой заработок в процентах:

In [236]:
all_data = all_data[all_data['ff_percents_day'].abs() > all_data['bi_percents_day']]
all_data['income_percent_day'] = abs(all_data['ff_percents_day'] + all_data['bi_percents_day'])
all_data['income_percent_year'] = all_data['income_percent_day'] * 366

В итоге для каждого инструмента получаем дневной и годовой заработок в столбцах *income_percent_day* и *income_percent_year* соответственно:

In [237]:
all_data

,index,symbol,bi_percents_day,bi_percents_year,ff_percents_day,ff_percents_year,income_percent_day,income_percent_year
2,18,ASTR,0.042894,15.656401,-0.044984,-16.419160,0.002090,0.764849
3,23,AXS,0.152473,55.652523,-0.186250,-67.981250,0.033777,12.362504
13,79,ICP,0.014491,5.289367,-0.026628,-9.719220,0.012137,4.441990
21,124,PERP,0.055417,20.227023,-0.073686,-26.895390,0.018270,6.686637
27,143,SPELL,0.042892,15.655458,-0.275696,-100.629040,0.232804,85.206386
28,146,STMX,0.106287,38.794588,-0.152168,-55.541320,0.045881,16.792614
29,156,TRB,4.138120,1510.413618,-6.925042,-2527.640330,2.786923,1020.013635
32,165,WLD,0.024921,9.096180,-0.031403,-11.462095,0.006482,2.372397


## Часть 4. Расчёт времени покрытия комиссий

Binance взимает с тейкеров со статусом VIP1 следующие комиссии:

In [238]:
futures_fee_taker = 0.04
margin_fee_taker = 0.1

С учётом этого добавляем в таблицу столбец, отражающий количество дней, необходимое для покрытия двух комиссий(покупка и продажа) в фьючерсах и марже:

In [239]:
all_data['Days_until_fee_is_covered'] = 2*(futures_fee_taker + margin_fee_taker) / all_data['income_percent_day']
all_data = all_data.sort_values(['Days_until_fee_is_covered'], ascending=True).reset_index(drop=True)

В столбце *Days_until_fee_is_covered* хранится количество дней, необходимое для покрытия комиссий:

In [240]:
all_data.head()

,index,symbol,bi_percents_day,bi_percents_year,ff_percents_day,ff_percents_year,income_percent_day,income_percent_year,Days_until_fee_is_covered
0,156,TRB,4.138120,1510.413618,-6.925042,-2527.64033,2.786923,1020.013635,0.100469
1,143,SPELL,0.042892,15.655458,-0.275696,-100.62904,0.232804,85.206386,1.202727
2,146,STMX,0.106287,38.794588,-0.152168,-55.54132,0.045881,16.792614,6.102683
3,23,AXS,0.152473,55.652523,-0.186250,-67.98125,0.033777,12.362504,8.289583
4,124,PERP,0.055417,20.227023,-0.073686,-26.89539,0.018270,6.686637,15.326090
